In [7]:
import numpy as np
import scipy as sp
import pyscf
from pyscf import gto, scf

mol = gto.Mole()
mol.atom = """
Li    0.0    0.0    0.0
Li    1.0    0.0    0.0
Li    0.5    0.866    0.0
"""
mol.basis = '6-31g'
mol.charge = 0
mol.spin = 3
mol.build()

In [8]:
mf = scf.UHF(mol)
mf.kernel()

converged SCF energy = -21.5185006073659  <S^2> = 3.7500162  2S+1 = 4.0000081


-21.518500607365937

In [8]:
mf.nelec

(6, 3)

In [ ]:
import pyscf.cc as cc

mycc = cc.uccsd.UCCSD(mf)
mycc.kernel()

# poggers this is close enough lmao

E(UCCSD) = -21.54413136016057  E_corr = -0.02563075279815653


(-0.025630752798156527,
 (array([[-4.61895787e-10,  4.42443799e-04,  3.34649370e-05,
          -4.67603682e-11, -1.52348468e-18, -2.98108203e-19,
          -1.43724020e-20, -3.49571752e-10, -7.19993459e-05,
           7.68494228e-12, -3.35225406e-04,  2.31397671e-10,
           1.85238411e-05, -7.73305810e-06,  1.32617107e-11,
           1.72128063e-18,  3.48609690e-19, -1.91148494e-03,
           3.15668618e-12,  2.94585420e-05, -2.88131633e-11],
         [-3.74796334e-09,  5.27989803e-05,  1.04374679e-04,
          -4.92643871e-11, -2.76315641e-17,  1.65576751e-17,
           1.29919327e-18, -3.13495148e-09,  5.35729433e-04,
          -9.40684890e-13, -5.27409784e-05,  1.97200803e-09,
          -3.12455903e-04, -7.09528103e-04, -2.75367793e-10,
           1.65283317e-18, -2.76367821e-18,  3.44172710e-05,
           1.45346468e-10,  1.21692121e-03,  2.07897894e-09],
         [-1.31677231e-03, -5.99364758e-10, -2.17348765e-10,
           1.33834223e-04, -1.47879509e-17,  9.87596051e-18

In [9]:
ccd_onUHF_bartlett(mf)

successful 1
successful 2
successful 3
successful 4
-0.02400686243826094
-0.024879719191436278
-0.02520565536022233
-0.025351975453270042
-0.0254231239848365
-0.025459333444374774
-0.02547837284695803
-0.025488655026628924
-0.025494334584095127
-0.025497532372018263
-0.02549936204868987
-0.025500423098326272
-0.025501045310308126
-0.02550141355630545
-0.025501633154173056
-0.025501764926293503
-0.025501844403625937
-0.02550189254226293
-0.025501921800884585
-0.02550193963541391
-0.025501950532331223
-0.025501957203556604


(-0.025501957203556604,
 array([[[[-0.00000000e+00,  1.07059444e-16,  2.39460030e-15, ...,
           -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
          [-1.07059433e-16, -0.00000000e+00, -7.07119595e-16, ...,
           -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
          [-2.39460025e-15,  7.07119857e-16, -0.00000000e+00, ...,
           -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
          ...,
          [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
           -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
          [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
           -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
          [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
           -0.00000000e+00, -0.00000000e+00, -0.00000000e+00]],
 
         [[ 9.82406371e-16, -1.48913573e-10, -3.79074039e-10, ...,
           -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
          [ 1.48913383e-10, -2.23566471e-17,  2.

In [ ]:
import numpy as np
import scipy as sp
import pyscf

def ccd_onUHF_bartlett(mf): # debug this

    Sovlp = mf.get_ovlp()
    h1e = mf.get_hcore()
    h2e = mf.mol.intor('int2e', aosym = "s1")




    # fock = mf.get_fock()

    norb = mf.mol.nao

    nelec_a = mf.nelec[0]
    nelec_b = mf.nelec[1]
    nelec = nelec_a + nelec_b

    nocc = nelec
    nvirt = 2*norb - nocc # unrestricted


    print("successful 1")

    coeffa = mf.mo_coeff[0]
    coeffb = mf.mo_coeff[1]


    # h1e_aa = np.einsum('ij,ia,jb->ab',h1e,coeffa, coeffa)
    # h1e_bb = np.einsum('ij,ia,jb->ab', h1e, coeffb, coeffb)

    # h1e_mo = np.zeros((norb*2, norb*2))
    # h1e_mo[:norb,:norb] = h1e_aa
    # h1e_mo[norb:,norb:] = h1e_bb



    h2e_aaaa = np.einsum('ijkl,ia,jb,kc,ld->abcd',h2e,coeffa,coeffa,coeffa,coeffa, optimize = True)
    h2e_bbbb = np.einsum('ijkl,ia,jb,kc,ld->abcd',h2e,coeffb,coeffb,coeffb,coeffb, optimize = True)

    h2e_aabb = np.einsum('ijkl,ia,jb,kc,ld->abcd',h2e,coeffa,coeffa,coeffb,coeffb, optimize = True)
    h2e_bbaa = np.einsum('ijkl,ia,jb,kc,ld->abcd',h2e,coeffb,coeffb,coeffa,coeffa, optimize = True)

    h2e_mo = np.zeros((norb*2, norb*2, norb*2, norb*2))
    h2e_mo[:norb,:norb,:norb,:norb] = h2e_aaaa
    h2e_mo[:norb,:norb,norb:,norb:] = h2e_aabb
    h2e_mo[norb:,norb:,:norb,:norb] = h2e_bbaa
    h2e_mo[norb:,norb:,norb:,norb:] = h2e_bbbb


    print("successful 2")


    nso = 2*norb

    idx_occ_alpha = np.arange(0, nelec_a)
    idx_occ_beta = np.arange(norb, norb+nelec_b)
    idx_occ = np.concatenate([idx_occ_alpha, idx_occ_beta])
    idx_all = np.arange(nso)
    idx_virt = np.setdiff1d(idx_all, idx_occ, assume_unique = True)

    perm = np.concatenate([idx_occ, idx_virt])

    # h1e_mo_sorted = h1e_mo[np.ix_(perm,perm)]
    h2e_mo_sorted = h2e_mo[np.ix_(perm,perm,perm,perm)]


    # now should be in terms of occupied, virtual for each of the indices.
    
    t2 = np.zeros((nocc,nocc,nvirt,nvirt)) # like ijab

    # initial guess is supposed to be the same as previous; <ij||ab>/Dijab


    h2e_mo_asym_chemist = h2e_mo_sorted - np.transpose(h2e_mo_sorted,(0,3,2,1)) # ai bj (normal) - aj bi (reshaped)

    # now transpose to physicist notation to match bartlett
    # a i b j --> a b i j; transpose(h2e_mo, (0, 2, 1, 3)
    h2e_mo_asym = np.transpose(h2e_mo_asym_chemist,(0, 2, 1, 3))  ## now these are < ab || ij >

    h2e_mo_asym_trunc = h2e_mo_asym[:nocc, :nocc, nocc:, nocc:]


    # t2 amplitudes antisymmetric upon pp/hh transformation
    # we need to generate our t2 guess

    # t2 has dimensions (i,j,a,b) = (occ, occ, vir, vir) = (nelec, nelec, nvirt, nvirt)

    print("successful 3")



    mo_energy_a = mf.mo_energy[0]
    mo_energy_b = mf.mo_energy[1]

    mo_energy = np.concatenate([mo_energy_a, mo_energy_b])
    mo_energy_sorted = mo_energy[np.ix_(perm)]

    denom = mo_energy_sorted[:nocc, None, None, None] + mo_energy_sorted[None, :nocc, None, None] - mo_energy_sorted[None, None, nocc:, None] - mo_energy_sorted[None, None, None, nocc:]

    t2 = h2e_mo_asym_trunc / denom

    print("successful 4")
    
    converged = False

    Eccd_prev = None

    while not converged:

        lhs = (h2e_mo_asym[:nocc, :nocc, nocc:, nocc:] + 0.5*np.einsum('abcd,ijcd->ijab',h2e_mo_asym[nocc:,nocc:,nocc:,nocc:],t2, optimize = True) + 
            0.5 * np.einsum('klij,klab->ijab',h2e_mo_asym[:nocc,:nocc,:nocc,:nocc],t2, optimize = True) + np.einsum('kbcj,ikac->ijab',h2e_mo_asym[:nocc,nocc:,nocc:,:nocc],t2, optimize = True) +
            np.einsum('kaci,jkbc->ijab',h2e_mo_asym[:nocc,nocc:,nocc:,:nocc],t2, optimize = True) - np.einsum('kacj,ikbc->ijab',h2e_mo_asym[:nocc,nocc:,nocc:,:nocc], t2, optimize = True) -
                np.einsum('kbci,jkac->ijab',h2e_mo_asym[:nocc,nocc:,nocc:,:nocc],t2, optimize = True) + 0.25 * np.einsum('klcd,ijcd,klab->ijab',h2e_mo_asym[:nocc,:nocc,nocc:,nocc:], t2, t2, optimize = True) +
                np.einsum('klcd,ikac,jlbd->ijab',h2e_mo_asym[:nocc,:nocc,nocc:,nocc:],t2,t2, optimize = True) - np.einsum('klcd,jkac,ilbd->ijab',h2e_mo_asym[:nocc,:nocc,nocc:,nocc:], t2, t2, optimize = True) - 
                0.5 * np.einsum('klcd,ikdc,ljab->ijab',h2e_mo_asym[:nocc,:nocc,nocc:,nocc:],t2,t2, optimize = True) + 0.5*np.einsum('klcd,jkdc,liab->ijab',h2e_mo_asym[:nocc,:nocc,nocc:,nocc:], t2, t2, optimize = True) -
                0.5 * np.einsum('klcd,lkac,ijdb->ijab',h2e_mo_asym[:nocc,:nocc,nocc:,nocc:], t2, t2, optimize = True) + 0.5*np.einsum('klcd,lkbc,ijda->ijab',h2e_mo_asym[:nocc,:nocc,nocc:,nocc:], t2, t2, optimize = True))
        
        t2 = lhs/denom

        Eccd = 0.25*np.einsum('ijab, ijab->',h2e_mo_asym[:nocc,:nocc,nocc:,nocc:], t2, optimize = True)

        if Eccd_prev is not None:
            if np.abs(Eccd - Eccd_prev) < 1e-8:
                converged = True

        Eccd_prev = Eccd

        print(Eccd)
    
    return Eccd, t2
